In [1]:
import mediapipe as mp
import numpy as np
import pandas as pd
import cv2
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2

from custom_pose_landmarks import CustomPoseLandmark

In [2]:
mp_drawing = solutions.drawing_utils
mp_pose = solutions.pose

custom = CustomPoseLandmark(mp_pose)

In [3]:
def prepare_dataframe():
    axes = ['X', 'Y', 'Z']
    names = custom.get_landmarks().values()
    column_names = [('_'.join([name, axis])).lower() for name in names for axis in axes]

    return pd.DataFrame(
        columns = ['timestamp'] + column_names,
        dtype = float
    )

In [4]:
def landmark2array(landmark):
    return np.array(
        [
            landmark.x,
            landmark.y,
            landmark.z,
            landmark.visibility
        ]
    )

def array2landmark(array):
    return landmark_pb2.NormalizedLandmark(
        x=array[0],
        y=array[1],
        z=array[2],
        visibility=array[3]
    )


def get_custom_landmarks(landmarks):
    """
    
    """
    # Create customize landmarks list
    custom_landmarks = landmark_pb2.NormalizedLandmarkList()

    # Extend list by chosen landmarks
    custom_landmarks.landmark.extend(
        [landmarks.landmark[index] for index in custom.values])

    # Calculate the coordinates of neck landmark
    left_shoulder = landmark2array(
        landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER])
    right_shoulder = landmark2array(
        landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER])
    thorax = np.mean([left_shoulder, right_shoulder], axis=0)
    neck_landmark = array2landmark(thorax)

    # Add neck landmark to custom list
    custom_landmarks.landmark.add().CopyFrom(neck_landmark)

    return custom_landmarks

In [5]:
source = r'C:/Mateusz/StrengthCoach/test_video.mp4'

In [11]:
data = prepare_dataframe()

time = 0
cap = cv2.VideoCapture(source)

# Setup MediaPipe instance
with mp_pose.Pose(
    min_detection_confidence=0.75,
    min_tracking_confidence=0.75,
    enable_segmentation=True
) as pose:
    while cap.isOpened():
        ret, image = cap.read()

        try:
            image_shape = image.shape
        
            # Recolor image for image processing
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Image MediaPipe processing -> detection
            results = pose.process(image)

            # Recolor back to BGR for visualization
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


            default_landmarks = results.pose_landmarks

            if default_landmarks:
                time += 1

                # Get custom landmarks and create new connection
                custom_landmarks = get_custom_landmarks(default_landmarks)

                # Prepare a single record storage
                record = np.array([time])
                
                for landmark in custom_landmarks.landmark:
                    # Extract pose landmarks coordinates and store as array
                    coordinates = landmark2array(landmark)[:3]
                    record = np.concatenate(
                        [
                            record,
                            coordinates
                        ]
                    )

                # Save pose landmarks coordinates in time as DataFrame
                data = pd.concat(
                    [data, pd.DataFrame([record], columns=data.columns)],
                    ignore_index=True
                )

        except:
            break

cap.release()
cv2.destroyAllWindows()

In [12]:
data

,timestamp,nose_x,nose_y,nose_z,left_shoulder_x,left_shoulder_y,left_shoulder_z,right_shoulder_x,right_shoulder_y,right_shoulder_z,...,right_ankle_z,left_foot_index_x,left_foot_index_y,left_foot_index_z,right_foot_index_x,right_foot_index_y,right_foot_index_z,thorax_x,thorax_y,thorax_z
0,1.0,0.476585,0.283651,-0.226193,0.593176,0.309569,0.067215,0.386084,0.316786,0.058415,...,-0.161846,0.645625,0.776603,-0.355386,0.405638,0.778760,-0.356290,0.489630,0.313177,0.062815
1,2.0,0.477664,0.253810,-0.114875,0.592562,0.309748,0.126231,0.386703,0.315315,0.088879,...,-0.141608,0.645534,0.777155,-0.351701,0.403889,0.779085,-0.328905,0.489633,0.312531,0.107555
2,3.0,0.477114,0.275805,-0.157181,0.592548,0.309754,0.113585,0.387788,0.315149,0.082285,...,-0.132630,0.645224,0.777289,-0.341084,0.403436,0.779273,-0.306473,0.490168,0.312452,0.097935
3,4.0,0.476946,0.276480,-0.170652,0.592321,0.309597,0.107555,0.388114,0.314668,0.075961,...,-0.136069,0.644564,0.777287,-0.341190,0.403018,0.779851,-0.314975,0.490218,0.312132,0.091758
4,5.0,0.476950,0.275201,-0.172617,0.592113,0.309343,0.100778,0.388204,0.313456,0.073906,...,-0.137029,0.644306,0.777292,-0.340825,0.403191,0.780147,-0.319415,0.490158,0.311399,0.087342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,719.0,0.508485,0.403946,-0.713777,0.569052,0.358011,-0.480179,0.391368,0.366410,-0.538601,...,-0.252146,0.540059,0.702126,0.162666,0.433865,0.752714,-0.444993,0.480210,0.362210,-0.509390
719,720.0,0.513977,0.404130,-0.666135,0.572334,0.359811,-0.464529,0.396641,0.366129,-0.517257,...,-0.219251,0.541938,0.701482,-0.019211,0.436039,0.750031,-0.411159,0.484488,0.362970,-0.490893
720,721.0,0.512469,0.406551,-0.784421,0.571712,0.358634,-0.485119,0.401008,0.364949,-0.577495,...,-0.219645,0.543110,0.699337,0.512282,0.442404,0.747106,-0.444438,0.486360,0.361791,-0.531307
721,722.0,0.514368,0.405033,-0.749017,0.571396,0.357923,-0.459312,0.403578,0.364288,-0.537416,...,-0.317998,0.540879,0.700048,0.314433,0.448318,0.744807,-0.564361,0.487487,0.361105,-0.498364
